### Jul AdventKalender D21

https://adventofcode.com/2022/day/21

#### Day 21.1 

Given a list of key-value pairs where key is the name of a monkey and value is either a number or an operation.For example:

    root: pppw + sjmn
    dbpl: 5
    cczh: sllz + lgvd
    zczc: 2
    ptdq: humn - dvpt
    dvpt: 3
    lfqf: 4
    humn: 5
    ljgn: 2
    sjmn: drzm * dbpl
    sllz: 4
    pppw: cczh / lfqf
    lgvd: ljgn * ptdq
    drzm: hmdt - zczc
    hmdt: 32
    
Work out the result of the monkey named **root**.

In [1]:
def readFile(file_name):
    data_monkeys = {}
    f = open(file_name, "r")
    while True:
        line = f.readline()
        if not line:
            break
        pair = line.strip().split(': ')
        name = pair[0]
        value = int(pair[1]) if pair[1].isnumeric() else pair[1].split(' ')
        data_monkeys[name]=value
    f.close()
    return data_monkeys

In [2]:
import operator
ops = { "+": operator.add, "-": operator.sub, "*": operator.mul , "/": operator.truediv }

def _operation(num1, op, num2):
    return ops[op](num1, num2)
  
def getNum(target, flip_func=None):
    global data_monkeys
    if target in data_monkeys: # part 1
        if isinstance(data_monkeys[target],int):
            return data_monkeys[target]
        target1, op, target2 = data_monkeys[target]
        data_monkeys[target] = _operation(getNum(target1, flip_func), op, getNum(target2, flip_func))
        return data_monkeys[target]
    else: # + part 2
        for name, value in data_monkeys.items():
            if isinstance(value,list) and target in value:
                key, value = flip_func(target, name, value)
                data_monkeys[key] = value
                del data_monkeys[name]
                break
        return getNum(target, flip_func)

In [3]:
data_monkeys = readFile('data/input21.txt')
getNum('root')

21120928600114.0

#### Day 21.2

A few changes:

* The correct operation for **monkey root** should be **=**, which means that it still listens for two numbers (from the same two monkeys as before), but now checks that the two numbers **match**.

* Find out the result of **humn** (the current value is not relevant now)

In [4]:
def _flipOps(key, name, value): # name = value(x1/key op x2/key)
    global data_monkeys
    x1, op, x2 = value
    if op == '+': # name = key + x2, or name = x1 + key => key = name - x1/x2
        res = [name, '-', x1] if key==x2 else [name, '-', x2]
    elif op == '*':
        res = [name, '/', x1] if key==x2 else [name, '/', x2]
    elif op == '-': # name = key - x2, or name = x1 - key => key = name + x2 / x1 - name
        res = [name, '+', x2] if key==x1 else [x1, '-', name]
    elif op == '/':
        res = [name, '*', x2] if key==x1 else [x1, '/', name]
    elif op == '=': # name = key = x2, or name = key = x1 => None, x1/x2
        res = data_monkeys[x2] if key==x1 else data_monkeys[x1]
    return key, res

In [5]:
data_monkeys = readFile('data/input21.txt')
data_monkeys['root'][1] = '='
del data_monkeys['humn']

In [6]:
getNum(target='humn', flip_func=_flipOps)

3453748220116.0